- remove activity related variables and added title connections and event_code diff
- use sorted in encode_title function to fix results by kernel

In [1]:
import pandas as pd
import numpy as np
import warnings
import datetime
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from sklearn import preprocessing
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, cohen_kappa_score
import lightgbm as lgb
from functools import partial
import json
import copy
import time
import seaborn as sns
import scipy as sp
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows",1000)
np.set_printoptions(precision=8)
warnings.filterwarnings("ignore")

In [2]:
def qwk(a1, a2):
    max_rat = 3
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)
    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))
    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)
    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)
    e = e / a1.shape[0]
    return np.round(1 - o / e, 8)

In [3]:
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])
        return -qwk(y, X_p)
    
    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')
        
    def predict(self, X, coef):
        return pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])

    def coefficients(self):
        return self.coef_['x']

# install

In [4]:
%%time
train = pd.read_csv('../input/data-science-bowl-2019/train.csv')
train_labels = pd.read_csv('../input/data-science-bowl-2019/train_labels.csv')
test = pd.read_csv('../input/data-science-bowl-2019/test.csv')
#specs = pd.read_csv('../input/data-science-bowl-2019/specs.csv')
sample_submission = pd.read_csv('../input/data-science-bowl-2019/sample_submission.csv')

CPU times: user 1min 16s, sys: 6.46 s, total: 1min 22s
Wall time: 1min 22s


In [5]:
keep_id = train[train.type == "Assessment"][['installation_id']].drop_duplicates()
train = pd.merge(train, keep_id, on="installation_id", how="inner")
train = train[train.installation_id.isin(train_labels.installation_id.unique())]

In [6]:
assess_title = ['Mushroom Sorter (Assessment)', 'Bird Measurer (Assessment)',
       'Cauldron Filler (Assessment)', 'Cart Balancer (Assessment)', 'Chest Sorter (Assessment)']
def remove_index_calc(df):
    additional_remove_index = []
    for i, session in df.groupby('installation_id', sort=False):
        last_row = session.index[-1]
        session = session[session.title.isin(assess_title)]
        first_row = session.index[-1] + 1
        for j in range(first_row, last_row+1):
            additional_remove_index.append(j)                
    return additional_remove_index
additional_remove_index = remove_index_calc(train)
train = train[~train.index.isin(additional_remove_index)]

# Preprocess and Feature engineering

In [7]:
%%time
def encode_title(train, test):
    train['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), train['title'], train['event_code']))
    test['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), test['title'], test['event_code']))
    list_of_title_eventcode = sorted(list(set(train['title_event_code'].unique()).union(set(test['title_event_code'].unique()))))

    list_of_user_activities = sorted(list(set(train['title'].unique()).union(set(test['title'].unique()))))
    list_of_event_code = sorted(list(set(train['event_code'].unique()).union(set(test['event_code'].unique()))))
    list_of_worlds = sorted(list(set(train['world'].unique()).union(set(test['world'].unique()))))
    activities_map = dict(zip(list_of_user_activities, np.arange(len(list_of_user_activities))))
    activities_labels = dict(zip(np.arange(len(list_of_user_activities)), list_of_user_activities))
    activities_world = dict(zip(list_of_worlds, np.arange(len(list_of_worlds))))
    assess_titles = sorted(list(set(train[train['type'] == 'Assessment']['title'].value_counts().index).union(set(test[test['type'] == 'Assessment']['title'].value_counts().index))))

    train['title'] = train['title'].map(activities_map)
    test['title'] = test['title'].map(activities_map)
    train['world'] = train['world'].map(activities_world)
    test['world'] = test['world'].map(activities_world)

    win_code = dict(zip(activities_map.values(), (4100*np.ones(len(activities_map))).astype('int')))
    win_code[activities_map['Bird Measurer (Assessment)']] = 4110
    
    train['timestamp'] = pd.to_datetime(train['timestamp'])
    test['timestamp'] = pd.to_datetime(test['timestamp'])
    
    train["misses"] = train["event_data"].apply(lambda x: json.loads(x)["misses"] if "\"misses\"" in x else np.nan)
    test["misses"] = test["event_data"].apply(lambda x: json.loads(x)["misses"] if "\"misses\"" in x else np.nan)
                
    return train, test, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, activities_world, list_of_title_eventcode

train, test, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, activities_world, list_of_title_eventcode = encode_title(train, test)

CPU times: user 38.4 s, sys: 3.09 s, total: 41.5 s
Wall time: 41.1 s


In [8]:
def get_data(user_sample, test_set=False):
    last_activity = 0
    user_activities_count = {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
    time_spent_each_act = {actv: 0 for actv in list_of_user_activities}
    title_eventcode_count = {str(ele): 0 for ele in list_of_title_eventcode}
    user_world_count = {"world_"+str(wor) : 0 for wor in activities_world.values()}
    
    last_session_time_sec = 0
    all_assessments = []
    accuracy_groups = {"0":0, "1":0, "2":0, "3":0}
    accumulated_accuracy_group = 0
    accumulated_correct_attempts = 0 
    accumulated_uncorrect_attempts = 0 
    accumulated_actions = 0
    counter = 0
    time_first_activity = float(user_sample['timestamp'].values[0])
    durations = []
    miss = 0
    crys_game_true = 0; crys_game_false = 0; total_crys_game_true = 0; total_crys_game_false = 0
    tree_game_true = 0; tree_game_false = 0; total_tree_game_true = 0; total_tree_game_false = 0
    magma_game_true = 0; magma_game_false = 0; total_magma_game_true = 0; total_magma_game_false = 0
    crys_game_acc = []; tree_game_acc = []; magma_game_acc = []
    t_crys_game_acc = []; t_tree_game_acc = []; t_magma_game_acc = []
    title_assessment_before = 0
    title_just_before = 0
        
    for i, session in user_sample.groupby('game_session', sort=False):      
        session_type = session['type'].iloc[0]
        session_title = session['title'].iloc[0]
        session_title_text = activities_labels[session_title]
        session_world = session["world"].iloc[0]
        
        # get current session time in seconds
        if session_type != 'Assessment':
            time_spent = int(session['game_time'].iloc[-1] / 1000)
            time_spent_each_act[activities_labels[session_title]] += time_spent   
            title_just_before = session_title
            
            if session_type == "Game":
                true = session['event_data'].str.contains('true').sum()
                false = session['event_data'].str.contains('false').sum() 
                if session_world == activities_world["CRYSTALCAVES"]:
                    crys_game_true += true
                    crys_game_false += false
                    total_crys_game_true += true
                    total_crys_game_false += false
                    crys_game_acc.append(true / (true + false) if (true + false) != 0 else 0)
                    t_crys_game_acc += crys_game_acc
                elif session_world == activities_world["TREETOPCITY"]:
                    tree_game_true += true
                    tree_game_false += false
                    total_tree_game_true += true
                    total_tree_game_false += false
                    tree_game_acc.append(true / (true + false) if (true + false) != 0 else 0)
                    t_tree_game_acc += tree_game_acc
                elif session_world == activities_world["MAGMAPEAK"]:
                    magma_game_true += true
                    magma_game_false += false
                    total_magma_game_true += true
                    total_magma_game_false += false
                    magma_game_acc.append(true / (true + false) if (true + false) != 0 else 0)
                    t_magma_game_acc += magma_game_acc
                else:
                    pass

        if (session_type == 'Assessment') & (test_set or len(session)>1): # test set or session in train_label
            all_attempts = session.query(f'event_code == {win_code[session_title]}')
            true_attempts = all_attempts['event_data'].str.contains('true').sum() # true in target assess
            false_attempts = all_attempts['event_data'].str.contains('false').sum() # false in target assessment
            
            # from start of installation_id to the start of target assessment ------------------------
            features = user_activities_count.copy() # appearance of each type without duplicates
            features.update(time_spent_each_act.copy()) # cumulative gameplay time in each title
            features.update(title_eventcode_count.copy()) # apperance of combi of title and event_code
            features.update(user_world_count.copy()) # appearance of world with duplicates
            features['accumulated_correct_attempts'] = accumulated_correct_attempts
            features['accumulated_uncorrect_attempts'] = accumulated_uncorrect_attempts
            accumulated_correct_attempts += true_attempts 
            accumulated_uncorrect_attempts += false_attempts
            features["misses"] = miss
            features['accumulated_actions'] = accumulated_actions
            if session_world == activities_world["CRYSTALCAVES"]:
                features["total_game_true"] = total_crys_game_true
                features["total_game_false"] = total_crys_game_false
                features['total_game_accuracy'] = total_crys_game_true / (total_crys_game_true + total_crys_game_false) if (total_crys_game_true + total_crys_game_false) != 0 else 0
                features["all_game_accuracy_std"] = np.std(t_crys_game_acc) if len(t_crys_game_acc) >=1 else 0
                features["game_true_bet_assess"] = crys_game_true
                features["game_false_bet_assess"] = crys_game_false
                features['game_accuracy_bet_assess'] = crys_game_true / (crys_game_true + crys_game_false) if (crys_game_true + crys_game_false) != 0 else 0
                features["game_accuracy_std_bet_assess"] = np.std(crys_game_acc) if len(crys_game_acc) >=1 else 0
                features["last_game_acc"] = crys_game_acc[-1] if len(crys_game_acc) >=1 else 0
            elif session_world == activities_world["TREETOPCITY"]:
                features["total_game_true"] = total_tree_game_true
                features["total_game_false"] = total_tree_game_false
                features['total_game_accuracy'] = total_tree_game_true / (total_tree_game_true + total_tree_game_false) if (total_tree_game_true + total_tree_game_false) != 0 else 0
                features["all_game_accuracy_std"] = np.std(t_tree_game_acc) if len(t_tree_game_acc) >=1 else 0
                features["game_true_bet_assess"] = tree_game_true
                features["game_false_bet_assess"] = tree_game_false
                features['game_accuracy_bet_assess'] = tree_game_true / (tree_game_true + tree_game_false) if (tree_game_true + tree_game_false) != 0 else 0
                features["game_accuracy_std"] = np.std(tree_game_acc) if len(tree_game_acc) >=1 else 0
                features["last_game_acc"] = tree_game_acc[-1] if len(tree_game_acc) >=1 else 0
            elif session_world == activities_world["MAGMAPEAK"]:
                features["total_game_true"] = total_magma_game_true
                features["total_game_false"] = total_magma_game_false
                features['total_game_accuracy'] = total_magma_game_true / (total_magma_game_true + total_magma_game_false) if (total_magma_game_true + total_magma_game_false) != 0 else 0
                features["all_game_accuracy_std"] = np.std(t_magma_game_acc) if len(t_magma_game_acc) >=1 else 0
                features["game_true_bet_assess"] = magma_game_true
                features["game_false_bet_assess"] = magma_game_false
                features['game_accuracy_bet_assess'] = magma_game_true / (magma_game_true + magma_game_false) if (magma_game_true + magma_game_false) != 0 else 0
                features["game_accuracy_std"] = np.std(magma_game_acc) if len(magma_game_acc) >=1 else 0
                features["last_game_acc"] = magma_game_acc[-1] if len(magma_game_acc) >=1 else 0
            crys_game_true = 0
            crys_game_false = 0
            tree_game_true = 0
            tree_game_false = 0
            magma_game_true = 0
            magma_game_false = 0
            crys_game_acc = []
            tree_game_acc = []
            magma_game_acc = []
                                    
            # unique type --------------------------------------------------------
            features['installation_id'] = session['installation_id'].iloc[-1]
            features['session_title'] = session_title
            
            # nums in target assessment data ------------------------------------------
            if durations == []: #span of timestamp in target assessment
                features['duration_mean'] = 0
                features['duration_std'] = 0
                features['last_duration'] = 0
                features['duration_max'] = 0
            else:
                features['duration_mean'] = np.mean(durations)
                features['duration_std'] = np.std(durations)
                features['last_duration'] = durations[-1]
                features['duration_max'] = np.max(durations)
            durations.append((session.iloc[-1, 2] - session.iloc[0, 2]).seconds) 
            
            accuracy = true_attempts/(true_attempts+false_attempts) if (true_attempts+false_attempts) != 0 else 0
            if accuracy == 0:
                features['accuracy_group'] = 0
            elif accuracy == 1:
                features['accuracy_group'] = 3
            elif accuracy == 0.5:
                features['accuracy_group'] = 2
            else:
                features['accuracy_group'] = 1
            features.update(accuracy_groups)
            accuracy_groups[str(features['accuracy_group'])] += 1
            features['accumulated_accuracy_group'] = accumulated_accuracy_group/counter if counter > 0 else 0
            accumulated_accuracy_group += features['accuracy_group']
            
            features['title*title_just_before'] = int(str(session['title'].iloc[0]) + str(title_just_before))
            title_just_before = session['title'].iloc[0]
            
            features['title_assessment_before'] = title_assessment_before
            features['title*title_assessment_before'] = int( str(session['title'].iloc[0]) + str(title_assessment_before) )
            title_assessment_before = session['title'].iloc[0]
                
            if test_set:
                all_assessments.append(features)
            elif true_attempts+false_attempts > 0:
                all_assessments.append(features)
                
            counter += 1
                        
        n_of_title_eventcode = Counter(session['title_event_code']) 
        for key in n_of_title_eventcode.keys():
            title_eventcode_count[str(key)] += n_of_title_eventcode[key]
            
        miss += np.sum(session["misses"])
            
        user_world_count["world_"+str(session_world)] += session.shape[0]

        accumulated_actions += len(session)
        if last_activity != session_type:
            user_activities_count[session_type] += 1
            last_activitiy = session_type
    if test_set:
        return all_assessments[-1]
    return all_assessments

In [9]:
new_train = []
for i, (ins_id, user_sample) in tqdm(enumerate(train.groupby('installation_id', sort=False)), total=train.installation_id.nunique(), desc='Installation_id', position=0):
    new_train += get_data(user_sample)
new_train = pd.DataFrame(new_train)
print(new_train.shape)
del train


(17690, 478)


In [10]:
new_test = []
for ins_id, user_sample in tqdm(test.groupby('installation_id', sort=False), total=test.installation_id.nunique(), desc='Installation_id', position=0):
    a = get_data(user_sample, test_set=True)
    new_test.append(a)   
new_test = pd.DataFrame(new_test)
print(new_test.shape)
del test


(1000, 478)


# Feature selection and modelling

In [11]:
X_train = new_train.drop(['accuracy_group'],axis=1) 
lbl = preprocessing.LabelEncoder()
lbl.fit(list(X_train["installation_id"]))
X_train["installation_id"] = lbl.transform(list(X_train["installation_id"]))
remove_features = []
for i in X_train.columns:
    if X_train[i].std() == 0 and i not in remove_features:
        remove_features.append(i)
X_train = X_train.drop(remove_features, axis=1)
X_train = X_train[sorted(X_train.columns.tolist())]
y_train = new_train.accuracy_group
print(X_train.shape)

(17690, 452)


In [12]:
n_folds=5
skf=StratifiedKFold(n_splits = n_folds)
coefficients = []
models = []
train_qwk_scores = []
test_qwk_scores = []
lgbm_params = {
    "objective" : "regression",
    "metric" : "rmse",
    "tree_learner": "serial",
    "max_depth" : 4,
    "boosting": 'gbdt',
    "num_leaves" : 13,
    "learning_rate" : 0.01,
}

#lgbm_params = {'boosting_type': 'gbdt', 
#              'metric': 'rmse', 
#              'objective': 'regression', 
#              'eval_metric': 'cappa', 
#              'n_jobs': -1, 
#              'seed': 42, 
#              'num_leaves': 26, 
#              'learning_rate': 0.017439684887749, 
#              'max_depth': 33, 
#              'lambda_l1': 3.27791989030057, 
#              'lambda_l2': 1.3047627805931334, 
#              'bagging_fraction': 0.896924978584253, 
#              'bagging_freq': 1, 
#              'colsample_bytree': 0.8710772167017853}

features_list = [i for i in X_train.columns if i != "installation_id"]
feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
    optR = OptimizedRounder()
    X_train2 = X_train.iloc[train_index,:]
    y_train2 = y_train.iloc[train_index]
    X_train2 = X_train2.drop(['installation_id'],axis=1)
    
    X_test2 = X_train.iloc[test_index,:]
    y_test2 = y_train.iloc[test_index]
    test2 = pd.concat([X_test2, y_test2], axis=1)
    test2 = test2.groupby('installation_id').apply(lambda x: x.sample(1, random_state=1223)).reset_index(drop=True)
    X_test2 = test2.drop(["accuracy_group", "installation_id"], axis=1)
    y_test2 = test2["accuracy_group"]
    
    lgb_train = lgb.Dataset(X_train2, y_train2)
    lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
    
    clf = lgb.train(
        lgbm_params, lgb_train,
        valid_sets=[lgb_train, lgb_eval],
        num_boost_round=100000,
        early_stopping_rounds=10,
    )
    
    models.append(clf)
    train_predict = clf.predict(X_train2, num_iteration = clf.best_iteration)
    test_predict = clf.predict(X_test2, num_iteration = clf.best_iteration)
    
    optR.fit(train_predict.reshape(-1,), y_train2)
    tmp_coefficients = optR.coefficients()
    print("fold_"+str(i)+" coefficients: ", tmp_coefficients)
    opt_train_preds = optR.predict(train_predict.reshape(-1, ), tmp_coefficients)
    train_qwk_score = qwk(y_train2, opt_train_preds)
    print("training qwk: ", train_qwk_score)
    opt_test_preds = optR.predict(test_predict.reshape(-1, ), tmp_coefficients)
    test_qwk_score = qwk(y_test2, opt_test_preds)
    print("validation qwk: ", test_qwk_score)
    train_qwk_scores.append(train_qwk_score)
    test_qwk_scores.append(test_qwk_score)
    coefficients.append(tmp_coefficients)
    
    feature_importance_df["Fold_"+str(i+1)] = clf.feature_importance()
    
feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]

print("                             ")
print("-----------------------------")
print('coefficients: ', coefficients)
print('train qwk list:', train_qwk_scores)
print('train qwk average score:',np.mean(train_qwk_scores))
print('valid qwk list: ', test_qwk_scores)
print('valid qwk average score:',np.mean(test_qwk_scores))

[1]	training's rmse: 1.25307	valid_1's rmse: 1.27304
Training until validation scores don't improve for 10 rounds
[2]	training's rmse: 1.2494	valid_1's rmse: 1.26957
[3]	training's rmse: 1.24582	valid_1's rmse: 1.26627
[4]	training's rmse: 1.24228	valid_1's rmse: 1.26289
[5]	training's rmse: 1.23882	valid_1's rmse: 1.25967
[6]	training's rmse: 1.23538	valid_1's rmse: 1.25653
[7]	training's rmse: 1.23202	valid_1's rmse: 1.25335
[8]	training's rmse: 1.2287	valid_1's rmse: 1.2503
[9]	training's rmse: 1.22546	valid_1's rmse: 1.24718
[10]	training's rmse: 1.22225	valid_1's rmse: 1.24423
[11]	training's rmse: 1.21913	valid_1's rmse: 1.24135
[12]	training's rmse: 1.21604	valid_1's rmse: 1.23849
[13]	training's rmse: 1.21302	valid_1's rmse: 1.23557
[14]	training's rmse: 1.21002	valid_1's rmse: 1.23282
[15]	training's rmse: 1.20711	valid_1's rmse: 1.23008
[16]	training's rmse: 1.20419	valid_1's rmse: 1.22743
[17]	training's rmse: 1.20133	valid_1's rmse: 1.22481
[18]	training's rmse: 1.19856	val

train qwk list: [0.60822112, 0.64801644, 0.63897551, 0.65015633, 0.66087027]
train qwk average score: 0.6412479340000001
valid qwk list:  [0.51432786, 0.57871011, 0.53040066, 0.51793534, 0.54943052]
valid qwk average score: 0.538160898

In [13]:
feature_importance_df.sort_values("Average", ascending=False).reset_index(drop=True).head(500)

,Feature,Fold_1,Fold_2,Fold_3,Fold_4,Fold_5,Average,Std,Cv
0,session_title,683,677,729,750,815,730.8,50.280811,0.068802
1,accumulated_accuracy_group,480,528,454,592,566,524.0,51.458721,0.098204
2,title*title_just_before,283,277,314,400,270,308.8,48.014165,0.155486
3,game_accuracy_std,322,244,286,335,274,292.2,32.902280,0.112602
4,Clip,248,298,278,296,340,292.0,29.959973,0.102603
5,misses,174,209,196,286,228,218.6,38.008420,0.173872
6,title*title_assessment_before,181,196,194,307,158,207.2,51.704545,0.249539
7,game_accuracy_std_bet_assess,86,161,226,224,246,188.6,58.711498,0.311302
8,Chow Time_4070,205,193,227,171,122,183.6,35.730099,0.194608
9,total_game_accuracy,145,132,127,254,224,176.4,52.316728,0.296580


In [14]:
print(X_train.shape)
print("")
print(", ".join('%s'%x for x in X_train.columns))

(17690, 452)

0, 1, 12 Monkeys_2000, 2, 3, Activity, Air Show, Air Show_2000, Air Show_2020, Air Show_2030, Air Show_2060, Air Show_2070, Air Show_2075, Air Show_3010, Air Show_3020, Air Show_3021, Air Show_3110, Air Show_3120, Air Show_3121, Air Show_4010, Air Show_4020, Air Show_4070, Air Show_4090, Air Show_4100, Air Show_4110, All Star Sorting, All Star Sorting_2000, All Star Sorting_2020, All Star Sorting_2025, All Star Sorting_2030, All Star Sorting_3010, All Star Sorting_3020, All Star Sorting_3021, All Star Sorting_3110, All Star Sorting_3120, All Star Sorting_3121, All Star Sorting_4010, All Star Sorting_4020, All Star Sorting_4030, All Star Sorting_4035, All Star Sorting_4070, All Star Sorting_4080, All Star Sorting_4090, All Star Sorting_4095, Assessment, Balancing Act_2000, Bird Measurer (Assessment)_2000, Bird Measurer (Assessment)_2010, Bird Measurer (Assessment)_2020, Bird Measurer (Assessment)_2030, Bird Measurer (Assessment)_3010, Bird Measurer (Assessment)_3020, Bird 

In [15]:
#import eli5
#from eli5.sklearn import PermutationImportance
#my_model = lgb.LGBMRegressor(**lgbm_params).fit(X_train2, y_train2)
#perm = PermutationImportance(my_model, random_state=1).fit(X_test2, y_test2)
#eli5.show_weights(perm, feature_names = X_test2.columns.tolist(), top = 500)

# prediction

In [16]:
X_test = new_test.drop(["installation_id", "accuracy_group"], axis=1)
X_test = X_test.drop(remove_features, axis=1)
X_test = X_test[sorted(X_test.columns.tolist())]
pred_value = np.zeros([X_test.shape[0]])
test_coefficients = np.mean(coefficients, axis=0)
for model in models:
    pred_value += model.predict(X_test, num_iteration = model.best_iteration) /n_folds
test_pred_class= optR.predict(pred_value.reshape(-1, ), test_coefficients)
sample_submission["accuracy_group"] = test_pred_class
sample_submission.to_csv('submission.csv', index=False)

In [17]:
sample_submission["accuracy_group"].value_counts(normalize = True)

3    0.461
2    0.306
1    0.118
0    0.115
Name: accuracy_group, dtype: float64